In [1]:
from tapenade.preprocessing import masked_gaussian_smooth_sparse
import tifffile
import numpy as np
from skimage.measure import regionprops
from tapenade.analysis.deformation import (
    add_tensor_inertia,
    add_true_strain_tensor,
    tensors_to_napari_vectors
)
from tapenade.analysis.deformation import (
    add_tensor_inertia,
    add_true_strain_tensor,
    tensors_to_napari_vectors
)

try: 
    import napari
    napari_available = True
except ImportError:
    napari_available = False
    print("Napari not available, skipping visualization")

# Multiscale analysis of deformation fields (inertia, true strain, etc.) 

### <font color='red'> After clicking on a cell, press "Shift+Enter" to run the code, or click on the "Run" button in the toolbar above.<br>

### Replace "..." signs with the appropriate path to your data.
</font>

The first step is to analyze each segmented instances from `labels` to compute the quantities of interest. We will use the `skimage.measure.regionprops` function to compute the centroid, inertia tensor, and true strain tensor of each segmented object. 

The inertia tensor is a 3x3 matrix that describes the distribution of mass in the object. The eigenvectors of the inertia tensor are the principal axes of the object, and the eigenvalues are the moments of inertia along these axes. The moments of inertia are related to the object's shape and can be used to compute the object's elongation, flatness, and orientation. Most notably, it is possible to transform the principal moments of inertia to get the principal lengths of the object, which are the lengths of the ellipsoid with the same moments of inertia as the object (the ellipse that best fits the object).

The principal length vector is not a quantity suited for local averaging among several objects. Instead, the inertia tensor of each object can be averaged, and a total average principal length vector can be computed.

However, the inertia tensor is not appropriate to quantify an average direction (e.g of collective alignment), as it over-represents the largest objects. Instead, we will use the true strain tensor, which is a 3x3 matrix whose eigen-values, derived from the principal lengths of the object, correspond to the logarithm of the amount of deformation with respect to a sphere of the same volume as the object.

### 1. Data loading

Load your data. If you do not have a mask, we recommend checking the `preprocessing` notebook to create one. If you don't have labels, you can use the `segmentation` notebook to create nuclei labels using StarDist3D.

In [ ]:
path_to_data = ...
path_to_mask = ...
path_to_labels = ...

data = tifffile.imread(path_to_data)
mask = tifffile.imread(path_to_mask)
labels = tifffile.imread(path_to_labels)

pixelsize = np.array([..., ..., ...]) # pixelsize in um/pixel
scale = 1/pixelsize

### 2. Building sparse arrays with the inertia and true strain tensors

We build spare arrays, i.e arrays of dimension `(n_points, n_dim space + n_dim points)` where `n_dim space` is the number of spatial dimensions and `n_dim points` is the number of dimensions of the signal at each point, e.g `n_dim points = 1` for a scalar field, `n_dim points = 3` for a 3D vector field, `n_dim points = 9` for a 3x3 matrix field, etc.

In [ ]:
# props is made of objects 'prop' with many morphological properties
# like volume, centroid, etc...
props = regionprops(labels) 

n_points = len(props)
n_dim_space = 3
n_dim_tensor = 9

sparse_inertia_tensor = np.zeros((n_points, n_dim_space+n_dim_tensor))
sparse_true_strain_tensor = np.zeros((n_points, n_dim_space+n_dim_tensor))

# store volumes to use later
volumes = np.zeros(n_points)

for index_label, prop in enumerate(props):
    add_tensor_inertia(prop, scale=scale)
    add_true_strain_tensor(prop, scale=scale)
    volumes[index_label] = prop.area

    sparse_inertia_tensor[index_label, :n_dim_space] = prop.centroid
    sparse_inertia_tensor[index_label, n_dim_space:] = prop.inertia_tensor.flatten()

    sparse_true_strain_tensor[index_label, :n_dim_space] = prop.centroid
    sparse_true_strain_tensor[index_label, n_dim_space:] = prop.true_strain_tensor.flatten()

### 3. Local averaging of the tensors

Specify a scale of analysis `sigma`, which correspond to the standard deviation of the Gaussian kernel used to average the tensors.

Smooth the sparse tensors signals to obtain the average tensors, defined at each centroid:

In [ ]:
sigma = 20

smoothed_sparse_inertia_tensor = masked_gaussian_smooth_sparse(
    sparse_inertia_tensor,
    is_temporal=False, # assuming you have loaded 3D data, not temporal TZYX data
    dim_space=n_dim_space,
    sigmas=sigma
)

smoothed_sparse_true_strain_tensor = masked_gaussian_smooth_sparse(
    sparse_true_strain_tensor,
    is_temporal=False,
    dim_space=n_dim_space,
    sigmas=sigma
)

The function `masked_gaussian_smooth_sparse` allows you to obtain a smoothed results defined at new sparse positions, e.g on a regular grid. 

Here we illustrate how to compute the averaged tensors on a regular grid of step 20 pixels:

In [ ]:
positions_on_grid = np.mgrid[
    [slice(0, dim, 20) for dim in labels.shape]
].reshape(labels.ndim, -1).T

# restrict to positions that are inside the mask of the sample
positions_on_grid = positions_on_grid[
    mask[positions_on_grid[:,0], positions_on_grid[:,1], positions_on_grid[:,2]]
]

smoothed_sparse_inertia_tensor_on_grid = masked_gaussian_smooth_sparse(
    sparse_inertia_tensor,
    is_temporal=False, # assuming you have loaded 3D data, not temporal TZYX data
    dim_space=n_dim_space,
    sigmas=sigma,
    positions=positions_on_grid
)

smoothed_sparse_true_strain_tensor_on_grid = masked_gaussian_smooth_sparse(
    sparse_true_strain_tensor,
    is_temporal=False,
    dim_space=n_dim_space,
    sigmas=sigma,
    positions=positions_on_grid
)

### 4. Quantifying deformation and alignment 

To quantify deformation and alignment, we extract the objects' principal length from the inertia tensors, and the main eigenvalue of the true strain tensors by diagonalizing the tensors.

Note that we extract the principal lengths only as a pedagogical example, as we have discussed at the beginning of the notebook that the principal lengths are not suited for quantifying collective alignment.

In [ ]:
inertia_vectors, inertia_angles = tensors_to_napari_vectors(
    smoothed_sparse_inertia_tensor_on_grid, 
    is_inertia_tensor=True, volumes=volumes, return_angles=True
)

true_strain_vectors, true_strain_angles = tensors_to_napari_vectors(
    smoothed_sparse_true_strain_tensor_on_grid, 
    is_inertia_tensor=False, return_angles=True
)

Vectors are formatted in an array `V` of shape `(n_objects, 2, dim_vector)`, where `V[i,0]` corresponds to the position at which the vector is defined, and `V[i,1]` corresponds to the vector itself.

The angles returned are the angles between the main eigenvector and the X axis.

We can do the same for our tensors computed on the regular grid:

In [ ]:
inertia_vectors_grid, inertia_angles_grid = tensors_to_napari_vectors(
    smoothed_sparse_inertia_tensor_on_grid,
    is_inertia_tensor=True, volumes=volumes, return_angles=True
)

true_strain_vectors_grid, true_strain_angles_grid = tensors_to_napari_vectors(
    smoothed_sparse_true_strain_tensor_on_grid,
    is_inertia_tensor=False, return_angles=True
)

### 5. Optional: Visualization with Napari

In [ ]:
if napari_available:

    viewer = napari.Viewer()

    viewer.add_image(data, name='data')
    viewer.add_labels(labels, name='labels')
    viewer.add_vectors(inertia_vectors, name='inertia_vectors',
                       properties={'angle': inertia_angles})
    viewer.add_vectors(true_strain_vectors, name='true_strain_vectors',
                       properties={'angle': true_strain_angles})
    viewer.add_vectors(inertia_vectors_grid, name='inertia_vectors_grid',
                       properties={'angle': inertia_angles_grid})
    viewer.add_vectors(true_strain_vectors_grid, name='true_strain_vectors_grid',
                       properties={'angle': true_strain_angles_grid})

    viewer.grid.enabled = True
    viewer.grid.shape = (3,2)

    napari.run()